In [ ]:
# Core
import sys
import pandas as pd
import numpy as np
from pathlib import Path

## Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

## Harp/Bonsai
sys.path.append('../../src/')
from bonsai import load_bonsai_config
load_bonsai_config(r"C:\git\AllenNeuralDynamics\aind-vr-foraging\Bonsai")
import harp
import harp.processing
import data_io

#Global Viz settings
sns.set_style('darkgrid') # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = 'Arial'

default_img_size = (15, 8)

In [ ]:
session_path = Path(r"Z:\scratch\vr-foraging\672107\20231006T124918")
session_path = Path(r"Z:\scratch\vr-foraging\672106\20231006T113138")
session_path = Path(r"Z:\scratch\vr-foraging\672103\20231006T112707")

# Harp Devices:
HarpBehavior = harp.HarpDevice("Behavior")
harp_behavior_data = data_io.HarpSource(device=HarpBehavior, path=session_path / "Behavior", name="behavior", autoload=False)

# Software Events
software_events = data_io.SoftwareEventSource(path=session_path / "SoftwareEvents", name="software_events")

In [ ]:
## Find changes in reward available
patches = software_events.streams.ActivePatch.data
reward = software_events.streams.GiveReward.data
reward_available = patches.iloc[0]["data"]["patchRewardFunction"]["initialRewardAmount"]

reward_updates = pd.concat([patches, reward])
reward_updates.sort_index(inplace=True)
reward_updates["currentReward"] = np.nan
for event in reward_updates.iterrows():
    if event[1]["name"] == 'GiveReward': #update reward
        reward_available -= event[1]["data"]
    elif event[1]["name"] == 'ActivePatch': #reset reward
        reward_available = event[1]["data"]["patchRewardFunction"]["initialRewardAmount"]
    else:
        raise ValueError("Unknown event type")
    reward_updates.at[event[0], "currentReward"] = reward_available


reward_updates

In [ ]:
# Find responses to Reward site
activeSite = software_events.streams.ActiveSite.data
choiceFeedback = software_events.streams.ChoiceFeedback.data
rewardSites = activeSite[activeSite["data"].apply(lambda x : x['label']) == 'Reward']
rewardSites["active_patch"] = -1
rewardSites["patch_label"] = ''

rewardSites["visit_number"] = -1
rewardSites["has_choice"] = False

current_patch_idx = -1
visit_number = 0
for idx,event in enumerate(rewardSites.iterrows()):
    arg_min, val_min = harp.processing.find_closest(event[0], patches.index.values, mode="below_zero")
    if not(np.isnan(arg_min)):
        rewardSites.loc[event[0], "active_patch"] = arg_min
    if current_patch_idx != arg_min:
        current_patch_idx = arg_min
        visit_number = 0
    else:
        visit_number += 1
    rewardSites.loc[event[0], "visit_number"] = visit_number

    if idx < len(rewardSites) -1:
        choice = choiceFeedback.loc[(choiceFeedback.index >= rewardSites.index[idx]) & (choiceFeedback.index < rewardSites.index[idx+1])]
    else:
        choice = choiceFeedback.loc[(choiceFeedback.index >= rewardSites.index[idx])]
    rewardSites.loc[event[0], "has_choice"] = len(choice) > 0

In [ ]:
rewardSites["patch_label"] = rewardSites["active_patch"].apply(lambda x : patches.iloc[x]["data"]["label"])
rewardSites

In [ ]:
session_duration = rewardSites.index[-1] - rewardSites.index[0]
threshold = rewardSites.index[0] + session_duration * 1
for patch_type, patch_type_df in rewardSites[rewardSites.index < threshold].groupby("patch_label"):
    plt.figure()
    plt.title(patch_type)
    choice_probability = np.zeros(len(patch_type_df["visit_number"].unique()))
    for visit_number, visit_number_df in patch_type_df.groupby("visit_number"):
        choice_probability[visit_number] = visit_number_df["has_choice"].sum() / len(visit_number_df)
    plt.plot(np.arange(0, choice_probability.shape[0]), choice_probability)
    plt.vlines(2, 0, 1, colors='r', linestyles='dashed')
    plt.xlabel("visit_number")
    plt.ylabel("choice_probability")